### Imports

In [39]:
import pandas as pd
import json

### Read dataframe

In [67]:
url = "https://raw.githubusercontent.com/dxli94/WLASL/master/start_kit/WLASL_v0.3.json"
df = pd.read_json(url)
df.head()

,gloss,instances
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra..."
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."


In [58]:
#Export a list of words in csv
#df["gloss"].to_csv("list_words.csv")

### Filter df only selected words

In [68]:
list_selected_words = ["headache", "cough", "sore throat", "blood", "pregnant", "diabetes", 
                       "stomach","pain", "allergy", "cold", "bone","diarrhea", "heart", "heart attack", 
                       "cochlear implant", "vomit", "depressed", "hurt", "infection", "tired", "thank you"]

df = df[df['gloss'].isin(list_selected_words)]
df

,gloss,instances
117,headache,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
147,cold,"[{'bbox': [129, 16, 476, 370], 'fps': 25, 'fra..."
175,tired,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
269,cough,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
287,heart,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
333,thank you,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
341,vomit,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
554,hurt,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
600,stomach,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
808,allergy,"[{'bbox': [170, 10, 492, 370], 'fps': 25, 'fra..."


In [69]:
print(f"Df shape: {df.shape}")

Df shape: (21, 2)


### Export pandas df to json file

In [70]:
new_json = df.to_json("WLASL_v0.3.json", orient='records')

### Explore instances

In [40]:
content = json.load(open("/Users/gehring/code/pariosur/sign-language-translator/scripts/WLASL_v0.3.json"))

id_videos = []
for entry in content:
    word = entry["gloss"]
    instances = entry['instances']

    for inst in instances:
        video_id = inst['video_id']
        id_videos.append([word, video_id])

In [41]:
id_list = pd.DataFrame(id_videos, columns=["word", "video_id"])
id_list

,word,video_id
0,headache,26832
1,headache,26835
2,headache,26836
3,headache,26837
4,headache,26838
...,...,...
227,heart attack,27019
228,heart attack,27020
229,heart attack,27021
230,heart attack,27022


#### Count videos per symptoms

In [42]:
id_list.groupby("word").count()

,video_id
word,
allergy,10
blood,9
bone,8
cochlear implant,9
cold,16
cough,14
depressed,8
diabetes,8
diarrhea,9


### Modelo